In [1]:
!pip install farm-haystack[inference] farm-haystack[metrics]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.4 MB/s eta 0:00:0031m25.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 29.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 KB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 KB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 33.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 KB 27.1 MB/s eta 0:00:00
  Prepa

In [2]:
from haystack.document_stores import InMemoryDocumentStore

In [3]:
document_store = InMemoryDocumentStore(use_bm25=True)

In [4]:
# Let's first fetch some tables that we want to query
# Here: 1000 tables from OTT-QA
from haystack.utils import fetch_archive_from_http

doc_dir = "data/tutorial15"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/table_text_dataset.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)


True

In [5]:
# Add the tables to the DocumentStore
import json
from haystack import Document
import pandas as pd


def read_tables(filename):
    processed_tables = []
    with open(filename) as tables:
        tables = json.load(tables)
        for key, table in tables.items():
            current_columns = table["header"]
            current_rows = table["data"]
            current_df = pd.DataFrame(columns=current_columns, data=current_rows)
            document = Document(content=current_df, content_type="table", id=key)
            processed_tables.append(document)

    return processed_tables


tables = read_tables(f"{doc_dir}/tables.json")
document_store.write_documents(tables, index="document")

# Showing content field and meta field of one of the Documents of content_type 'table'
print(tables[0].content)
print(tables[0].meta)


Updating BM25 representation...: 100%|██████████| 437/437 [00:00<00:00, 15016.35 docs/s]

                Opponent    M    W    L  T  NR   Win% First  Last
0            Afghanistan    2    2    0  0   0  100.0  2012  2014
1              Australia   98   32   62  1   3  34.21  1975  2017
2             Bangladesh   35   31    4  0   0  88.57  1986  2015
3                 Canada    2    2    0  0   0  100.0  1979  2011
4                England   82   31   49  0   2  38.75  1974  2017
5              Hong Kong    2    2    0  0   0  100.0  2004  2008
6                  India  129   73   52  0   4   58.4  1978  2017
7                Ireland    7    5    1  1   0  78.57  2007  2016
8                  Kenya    6    6    0  0   0  100.0  1996  2011
9                Namibia    1    1    0  0   0  100.0  2003  2003
10           Netherlands    3    3    0  0   0  100.0  1996  2003
11           New Zealand  103   53   47  1   2  52.97  1973  2018
12              Scotland    3    3    0  0   0  100.0  1999  2013
13          South Africa   73   25   47  0   1  34.72  1992  2017
14        

In [6]:
from haystack.nodes.retriever import EmbeddingRetriever

retriever = EmbeddingRetriever(document_store=document_store, embedding_model="deepset/all-mpnet-base-v2-table")


/home/ryan/github/generative-ai/venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
# Add table embeddings to the tables in DocumentStore
document_store.update_embeddings(retriever=retriever)


Updating Embedding:   0%|          | 0/437 [00:00<?, ? docs/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Documents Processed: 10000 docs [03:58, 41.91 docs/s]          


In [8]:
# Try the Retriever
retrieved_tables = retriever.retrieve("Who won the Super Bowl?", top_k=5)

# Get highest scored table
print(retrieved_tables[0].content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                     Year                   Coach              Super Bowl  \
0                    1966          Vince Lombardi                       I   
1                    1967          Vince Lombardi                      II   
2                    1996           Mike Holmgren                    XXXI   
3                    2010           Mike McCarthy                     XLV   
4  Total Super Bowls won:  Total Super Bowls won:  Total Super Bowls won:   

                  Location                Opponent  Score Record  
0  Los Angeles, California      Kansas City Chiefs  35–10   12–2  
1           Miami, Florida         Oakland Raiders  33–14  9–4–1  
2   New Orleans, Louisiana    New England Patriots  35–21   13–3  
3         Arlington, Texas     Pittsburgh Steelers  31–25   10–6  
4   Total Super Bowls won:  Total Super Bowls won:      4      4  


In [9]:
from haystack.nodes import TableReader

reader = TableReader(model_name_or_path="google/tapas-base-finetuned-wtq", max_seq_len=512)


In [10]:
# Try the TableReader on one Table

table_doc = document_store.get_document_by_id("36964e90-3735-4ba1-8e6a-bec236e88bb2")
print(table_doc.content)


                  Name        Program           Role       Salary     Year  \
0         Simon Cowell   The X Factor          Judge  $75 million  2012–13   
1       Britney Spears  American Idol    $25 million      2017–18     [15]   
2       Jennifer Lopez    $20 million        2011–12         [16]      nan   
3         Mariah Carey    $18 million        2012–13         [17]      nan   
4          Hugh Laurie          House  Gregory House  $15 million     2013   
5        Ryan Seacrest  American Idol           Host      2013–16     [14]   
6           Katy Perry   The X Factor          Judge         2012     [17]   
7          Miley Cyrus      The Voice          Coach  $13 million  2016–17   
8          Adam Levine        2016–18           [18]          nan      nan   
9        Blake Shelton        2016–18           [18]          nan      nan   
10  Christina Aguilera  $12.5 million           2013         [19]      nan   
11      Kelly Clarkson    $12 million           2018         [20

In [11]:
from haystack.utils import print_answers

prediction = reader.predict(query="Who played Gregory House in the series House?", documents=[table_doc])
print_answers(prediction, details="all")


'Query: Who played Gregory House in the series House?'
'Answers:'
[   <Answer {'answer': 'Hugh Laurie', 'type': 'extractive', 'score': 1.0, 'context': [['Name', 'Program', 'Role', 'Salary', 'Year', 'Ref.'], ['Simon Cowell', 'The X Factor', 'Judge', '$75 million', '2012–13', '[14]'], ['Britney Spears', 'American Idol', '$25 million', '2017–18', '[15]', 'nan'], ['Jennifer Lopez', '$20 million', '2011–12', '[16]', 'nan', 'nan'], ['Mariah Carey', '$18 million', '2012–13', '[17]', 'nan', 'nan'], ['Hugh Laurie', 'House', 'Gregory House', '$15 million', '2013', '[14]'], ['Ryan Seacrest', 'American Idol', 'Host', '2013–16', '[14]', 'nan'], ['Katy Perry', 'The X Factor', 'Judge', '2012', '[17]', 'nan'], ['Miley Cyrus', 'The Voice', 'Coach', '$13 million', '2016–17', '[18]'], ['Adam Levine', '2016–18', '[18]', 'nan', 'nan', 'nan'], ['Blake Shelton', '2016–18', '[18]', 'nan', 'nan', 'nan'], ['Christina Aguilera', '$12.5 million', '2013', '[19]', 'nan', 'nan'], ['Kelly Clarkson', '$12 million', '2

In [12]:
print(f"Predicted answer: {prediction['answers'][0].answer}")
print(f"Meta field: {prediction['answers'][0].meta}")


Predicted answer: Hugh Laurie
Meta field: {'aggregation_operator': 'NONE', 'answer_cells': ['Hugh Laurie']}


In [13]:
# Initialize pipeline
from haystack import Pipeline

table_qa_pipeline = Pipeline()
table_qa_pipeline.add_node(component=retriever, name="EmbeddingRetriever", inputs=["Query"])
table_qa_pipeline.add_node(component=reader, name="TableReader", inputs=["EmbeddingRetriever"])


In [14]:
prediction = table_qa_pipeline.run("When was Guilty Gear Xrd : Sign released?", params={"top_k": 30})
print_answers(prediction, details="minimum")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Query: When was Guilty Gear Xrd : Sign released?'
'Answers:'
[   {   'answer': 'December 16 , 2014',
        'context':                                         Title       First release  \
0                      Guilty Gear Xrd : Sign  December 16 , 2014   
1          BlazBlue : Chrono Phantasma Extend      June 30 , 2015   
2         Aegis of Earth : Protonovus Assault     March 15 , 2016   
3                  BlazBlue : Central Fiction    October 6 , 2016   
4     Chronicles of Teddy : Harmony of Exidus     March 29 , 2016   
5                 Guilty Gear Xrd : Revelator       June 7 , 2016   
6   Exist Archive : The Other Side of the Sky   October 18 , 2016   
7                     Guilty Gear Xrd : Rev 2       May 25 , 2017   
8      Under Night In-Birth Exe : Late [ st ]           Late 2017   
9                   School Girl/Zombie Hunter           Late 2017   
10                           Tokyo Xanadu eX+                2017   
11       Code : Realize ~Bouquet of Rainbows~      

In [15]:
# Add 500 text passages to our document store.


def read_texts(filename):
    processed_passages = []
    with open(filename) as passages:
        passages = json.load(passages)
        for key, content in passages.items():
            document = Document(content=content, content_type="text", id=key)
            processed_passages.append(document)

    return processed_passages


passages = read_texts(f"{doc_dir}/texts.json")
document_store.write_documents(passages, index="documents")


Updating BM25 representation...:   0%|          | 0/563 [00:00<?, ? docs/s]

Updating BM25 representation...: 100%|██████████| 563/563 [00:00<00:00, 30572.55 docs/s]


In [16]:
document_store.update_embeddings(retriever=retriever, update_existing_embeddings=False)


Updating Embedding:   0%|          | 0/437 [00:00<?, ? docs/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Documents Processed: 10000 docs [03:52, 43.08 docs/s]          


In [17]:
from haystack.nodes import FARMReader, RouteDocuments, JoinAnswers

text_reader = FARMReader("deepset/roberta-base-squad2")
# In order to get meaningful scores from the TableReader, use "deepset/tapas-large-nq-hn-reader" or
# "deepset/tapas-large-nq-reader" as TableReader models. The disadvantage of these models is, however,
# that they are not capable of doing aggregations over multiple table cells.
table_reader = TableReader("deepset/tapas-large-nq-hn-reader")
route_documents = RouteDocuments()
join_answers = JoinAnswers()


In [18]:
text_table_qa_pipeline = Pipeline()
text_table_qa_pipeline.add_node(component=retriever, name="EmbeddingRetriever", inputs=["Query"])
text_table_qa_pipeline.add_node(component=route_documents, name="RouteDocuments", inputs=["EmbeddingRetriever"])
text_table_qa_pipeline.add_node(component=text_reader, name="TextReader", inputs=["RouteDocuments.output_1"])
text_table_qa_pipeline.add_node(component=table_reader, name="TableReader", inputs=["RouteDocuments.output_2"])
text_table_qa_pipeline.add_node(component=join_answers, name="JoinAnswers", inputs=["TextReader", "TableReader"])


In [19]:
# Remove the following comment to generate the structure of the combined Table an Text QA pipeline.
# text_table_qa_pipeline.draw("pipeline.png")


In [20]:
# Example query whose answer resides in a text passage
predictions = text_table_qa_pipeline.run(query="Who was Thomas Alva Edison?")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
# We can see both text passages and tables as contexts of the predicted answers.
print_answers(predictions, details="minimum")


'Query: Who was Thomas Alva Edison?'
'Answers:'
[   {   'answer': 'Ian Nelson',
        'context':                         Name  \
0                Woody Allen   
1           Rachel Brosnahan   
2   Vinnette Justine Carroll   
3                 Cy Coleman   
4               Tony Kushner   
5                  Ami Miron   
6             Harvey Leonard   
7             Vince Gilligan   
8                 Ian Nelson   
9             Jonathan Meath   
10             Debra Messing   
11             David Milhous   
12            Camryn Manheim   
13            Terrance Moran   
14            Jeffrey Wright   
15             Paul Tazewell   
16               Aziz Ansari   
17           Rachel Attridge   
18         Sterling K. Brown   
19            Jeremy Tchaban   

                                      Relation to NYU  \
0                                          TSOA 1953*   
1                                   TSOA 2012 , B.F.A   
2                                     GSAS 1946 , M.A   


In [22]:
# Example query whose answer resides in a table
predictions = text_table_qa_pipeline.run(query="Which country does the film Macaroni come from?")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
# We can see both text passages and tables as contexts of the predicted answers.
print_answers(predictions, details="minimum")


'Query: Which country does the film Macaroni come from?'
'Answers:'
[   {   'answer': 'Italian',
        'context':    Submitting country Film title used in nomination        Language ( s )  \
0           Argentina            The Official Story               Spanish   
1             Austria                       Malambo                German   
2             Belgium                          Dust                French   
3              Canada          Jacques and November                French   
4      Czechoslovakia              Scalpel , Please                 Czech   
5             Denmark               Twist and Shout                Danish   
6              France        Three Men and a Cradle                French   
7        West Germany                 Angry Harvest                German   
8             Hungary                  Colonel Redl                German   
9             Iceland                   Deep Winter             Icelandic   
10              India                

In [24]:
from haystack import Label, MultiLabel, Answer


def read_labels(filename, tables):
    processed_labels = []
    with open(filename) as labels:
        labels = json.load(labels)
        for table in tables:
            if table.id not in labels:
                continue
            label = labels[table.id]
            label = Label(
                query=label["query"],
                document=table,
                is_correct_answer=True,
                is_correct_document=True,
                answer=Answer(answer=label["answer"]),
                origin="gold-label",
            )
            processed_labels.append(MultiLabel(labels=[label]))
    return processed_labels


table_labels = read_labels(f"{doc_dir}/labels.json", tables)
passage_labels = read_labels(f"{doc_dir}/labels.json", passages)


In [25]:
eval_results = text_table_qa_pipeline.eval(table_labels + passage_labels, params={"top_k": 10})


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
# Calculating and printing the evaluation metrics
print(eval_results.calculate_metrics())


{'EmbeddingRetriever': {'recall_multi_hit': 0.3, 'recall_single_hit': 0.3, 'precision': 0.030000000000000006, 'map': 0.25833333333333336, 'mrr': 0.25833333333333336, 'ndcg': 0.2678103593554011}, 'TextReader': {'exact_match': 0.0, 'f1': 0.0, 'num_examples_for_eval': 20.0}, 'TableReader': {'exact_match': 0.05, 'f1': 0.11285714285714285, 'num_examples_for_eval': 20.0}, 'JoinAnswers': {'exact_match': 0.05, 'f1': 0.11285714285714285, 'num_examples_for_eval': 20.0}}
